In [5]:
import sys
import time
import datetime

In [6]:
TOPIC_Step2_NAME="Sahamyab-Tweets2"
KAFKA_SERVER="kafka-broker:29092"

In [7]:
import os
# https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html

# setup arguments
os.environ['PYSPARK_SUBMIT_ARGS']='--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 pyspark-shell'

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Step2_6-SQL-Queries") \
    .config("spark.executor.memory", "1024mb") \
    .config("spark.executor.cores","1") \
    .config("spark.cores.max", "1") \
    .config("spark.sql.session.timeZone", "Asia/Tehran") \
    .getOrCreate()    
    
    

In [8]:
spark.sparkContext.setLogLevel("ERROR")


In [9]:
# https://sparkbyexamples.com/spark/spark-sql-structtype-on-dataframe/
schema = StructType([StructField("id", StringType(), True),\
                         StructField("content", StringType(), True),\
                         StructField("sendTime", StringType(), True),\
                         StructField("sendTimePersian", StringType(), True),\
                         StructField("senderName", StringType(), True),\
                         StructField("senderUsername", StringType(), True),\
                         StructField("type", StringType(), True),\
                         StructField("hashtags", ArrayType(StringType()), True)
                    ])

In [10]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_SERVER) \
  .option("subscribe", TOPIC_Step2_NAME) \
  .option("startingOffsets", "earliest") \
  .load()

In [11]:
tweetsStringDF = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

tweetsDF = tweetsStringDF.select(from_json(col("value"), schema).alias("data")).select("data.*")
tweetsDF = tweetsDF.withColumn("timestamp", unix_timestamp("sendTime", "yyyy-MM-dd'T'HH:mm:ss'Z'").cast('timestamp')) \
            .withColumn("persian_timestamp", from_utc_timestamp("timestamp", "Asia/Tehran").cast('timestamp')) \
            .withColumn("persianYear", tweetsDF['sendTimePersian'].substr(0, 4)) \
            .withColumn("persianMonth", tweetsDF['sendTimePersian'].substr(6, 2)) \
            .withColumn("persianDay", tweetsDF['sendTimePersian'].substr(9, 2))


In [13]:
tweetsDF.createOrReplaceTempView("Tweets")


In [10]:

New_DF = spark.sql("SELECT senderUsername, senderName,persianYear, persianMonth, persianDay,type, Hashtags \
                                    FROM Tweets \
                                    WHERE persianYear = '1400'")


# Write out our dataframe to the console
query = New_DF.writeStream\
                      .outputMode("append")\
                      .format("console")\
                      .option("truncate", "false")\
                      .option("numRows", 30)\
                      .start()\
                      .awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+--------------+----------+-----------+------------+----------+-----+--------+
|senderUsername|senderName|persianYear|persianMonth|persianDay|type |Hashtags|
+--------------+----------+-----------+------------+----------+-----+--------+
|sftt          |نیمه سوم  |1400       |07          |16        |quote|[]      |
|sarvarzadeh   |SHS       |1400       |07          |16        |twit |[فملی]  |
|hassanferi100 |همای      |1400       |07          |16        |quote|[]      |
|mohammad7543  |Mohammad  |1400       |07          |16        |twit |[پالایش]|
|arashii       |ARASH     |1400       |07          |16        |twit |[کالا]  |
|capitiz       |capitiz   |1400       |07          |16        |twit |[]      |
|mir48833      |مازیار    |1400       |07          |16        |twit |[کاما]  |
|parhambolet   |صدای بورس |1400       |07          |16        |quote|[دست]   |
|rezarfatia    |Reza125   |1400   

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------+----------+-----------+------------+----------+----+--------+
|senderUsername|senderName|persianYear|persianMonth|persianDay|type|Hashtags|
+--------------+----------+-----------+------------+----------+----+--------+
|mohammad_17799|محمد باجو |1400       |07          |16        |twit|[کنور]  |
+--------------+----------+-----------+------------+----------+----+--------+

-------------------------------------------
Batch: 3
-------------------------------------------
+---------------+---------------+-----------+------------+----------+------+--------+
|senderUsername |senderName     |persianYear|persianMonth|persianDay|type  |Hashtags|
+---------------+---------------+-----------+------------+----------+------+--------+
|agooly         |agooly         |1400       |07          |16        |quote |[شلرد]  |
|abo09906       |عباس           |1400       |07          |16        |qu

-------------------------------------------
Batch: 4
-------------------------------------------


+--------------+----------+-----------+------------+----------+-----+--------+
|senderUsername|senderName|persianYear|persianMonth|persianDay|type |Hashtags|
+--------------+----------+-----------+------------+----------+-----+--------+
|mohammad_17799|محمد باجو |1400       |07          |16        |twit |[کنور]  |
|agooly        |agooly    |1400       |07          |16        |quote|[شلرد]  |
+--------------+----------+-----------+------------+----------+-----+--------+

-------------------------------------------
Batch: 5
-------------------------------------------
+---------------+---------------+-----------+------------+----------+------+--------+
|senderUsername |senderName     |persianYear|persianMonth|persianDay|type  |Hashtags|
+---------------+---------------+-----------+------------+----------+------+--------+
|abo09906       |عباس           |1400       |07          |16        |quote |[]      |
|tavakolizade   |مهدی توکلی زاده|1400       |07          |16        |twit  |[زفکا]  

-------------------------------------------
Batch: 6
-------------------------------------------
+--------------+----------+-----------+------------+----------+----+--------+
|senderUsername|senderName|persianYear|persianMonth|persianDay|type|Hashtags|
+--------------+----------+-----------+------------+----------+----+--------+
|mohammad_17799|محمد باجو |1400       |07          |16        |twit|[کنور]  |
+--------------+----------+-----------+------------+----------+----+--------+

-------------------------------------------
Batch: 7
-------------------------------------------
+---------------+---------------+-----------+------------+----------+------+--------+
|senderUsername |senderName     |persianYear|persianMonth|persianDay|type  |Hashtags|
+---------------+---------------+-----------+------------+----------+------+--------+
|agooly         |agooly         |1400       |07          |16        |quote |[شلرد]  |
|abo09906       |عباس           |1400       |07          |16        |qu

-------------------------------------------
Batch: 8
-------------------------------------------
+--------------+----------+-----------+------------+----------+-----+--------+
|senderUsername|senderName|persianYear|persianMonth|persianDay|type |Hashtags|
+--------------+----------+-----------+------------+----------+-----+--------+
|mohammad_17799|محمد باجو |1400       |07          |16        |twit |[کنور]  |
|agooly        |agooly    |1400       |07          |16        |quote|[شلرد]  |
+--------------+----------+-----------+------------+----------+-----+--------+

-------------------------------------------
Batch: 9
-------------------------------------------
+---------------+---------------+-----------+------------+----------+------+--------+
|senderUsername |senderName     |persianYear|persianMonth|persianDay|type  |Hashtags|
+---------------+---------------+-----------+------------+----------+------+--------+
|abo09906       |عباس           |1400       |07          |16        |quot

-------------------------------------------
Batch: 10
-------------------------------------------
+--------------+----------+-----------+------------+----------+----+--------+
|senderUsername|senderName|persianYear|persianMonth|persianDay|type|Hashtags|
+--------------+----------+-----------+------------+----------+----+--------+
|mohammad_17799|محمد باجو |1400       |07          |16        |twit|[کنور]  |
+--------------+----------+-----------+------------+----------+----+--------+

-------------------------------------------
Batch: 11
-------------------------------------------
+---------------+---------------+-----------+------------+----------+------+--------+
|senderUsername |senderName     |persianYear|persianMonth|persianDay|type  |Hashtags|
+---------------+---------------+-----------+------------+----------+------+--------+
|agooly         |agooly         |1400       |07          |16        |quote |[شلرد]  |
|abo09906       |عباس           |1400       |07          |16        |

-------------------------------------------
Batch: 12
-------------------------------------------
+--------------+----------+-----------+------------+----------+----+--------+
|senderUsername|senderName|persianYear|persianMonth|persianDay|type|Hashtags|
+--------------+----------+-----------+------------+----------+----+--------+
|mohammad_17799|محمد باجو |1400       |07          |16        |twit|[کنور]  |
+--------------+----------+-----------+------------+----------+----+--------+

-------------------------------------------
Batch: 13
-------------------------------------------
+---------------+---------------+-----------+------------+----------+------+--------+
|senderUsername |senderName     |persianYear|persianMonth|persianDay|type  |Hashtags|
+---------------+---------------+-----------+------------+----------+------+--------+
|agooly         |agooly         |1400       |07          |16        |quote |[شلرد]  |
|abo09906       |عباس           |1400       |07          |16        |

KeyboardInterrupt: 

In [ ]:
New_DF.stop()

In [ ]:
User_Activity_DF = spark.sql("SELECT senderUsername, count(*) as UserActivityCount \
                    FROM Tweets\
                    WHERE persianYear='1400' \
                    GROUP BY senderUsername\
                    order by UserActivityCount desc \
                ")

User_Activity_DF.createOrReplaceTempView("UserTweetCounts")


# Write out our dataframe to the console
query = User_Activity_DF.writeStream \
                .outputMode("complete")\
                .format("console")\
                .option("truncate", "false")\
                .option("numRows", 10)\
                .start()\
                .awaitTermination()

-------------------------------------------
Batch: 51
-------------------------------------------
+--------------+----------+-----------+------------+----------+----+--------+
|senderUsername|senderName|persianYear|persianMonth|persianDay|type|Hashtags|
+--------------+----------+-----------+------------+----------+----+--------+
|mohammad_17799|محمد باجو |1400       |07          |16        |twit|[کنور]  |
+--------------+----------+-----------+------------+----------+----+--------+



-------------------------------------------
Batch: 52
-------------------------------------------
+---------------+---------------+-----------+------------+----------+------+--------+
|senderUsername |senderName     |persianYear|persianMonth|persianDay|type  |Hashtags|
+---------------+---------------+-----------+------------+----------+------+--------+
|agooly         |agooly         |1400       |07          |16        |quote |[شلرد]  |
|abo09906       |عباس           |1400       |07          |16        |quote |[]      |
|tavakolizade   |مهدی توکلی زاده|1400       |07          |16        |twit  |[زفکا]  |
|hm8120775447890|گمنام          |1400       |07          |15        |retwit|[دی]    |
|amir2489       |امیر           |1400       |07          |16        |twit  |[غبهار] |
|measam313      |MA313          |1400       |07          |16        |twit  |[برکت]  |
|alis3394       |محمدعلی صادقی  |1400       |07          |16        |quote |[]      |
|matin2930      |اقتصاد فردا    |1400     

-------------------------------------------
Batch: 53
-------------------------------------------
+--------------+----------+-----------+------------+----------+----+--------+
|senderUsername|senderName|persianYear|persianMonth|persianDay|type|Hashtags|
+--------------+----------+-----------+------------+----------+----+--------+
|mohammad_17799|محمد باجو |1400       |07          |16        |twit|[کنور]  |
+--------------+----------+-----------+------------+----------+----+--------+



-------------------------------------------
Batch: 54
-------------------------------------------
+---------------+---------------+-----------+------------+----------+------+--------+
|senderUsername |senderName     |persianYear|persianMonth|persianDay|type  |Hashtags|
+---------------+---------------+-----------+------------+----------+------+--------+
|agooly         |agooly         |1400       |07          |16        |quote |[شلرد]  |
|abo09906       |عباس           |1400       |07          |16        |quote |[]      |
|tavakolizade   |مهدی توکلی زاده|1400       |07          |16        |twit  |[زفکا]  |
|hm8120775447890|گمنام          |1400       |07          |15        |retwit|[دی]    |
|amir2489       |امیر           |1400       |07          |16        |twit  |[غبهار] |
|measam313      |MA313          |1400       |07          |16        |twit  |[برکت]  |
|alis3394       |محمدعلی صادقی  |1400       |07          |16        |quote |[]      |
|matin2930      |اقتصاد فردا    |1400     

-------------------------------------------
Batch: 55
-------------------------------------------
+--------------+----------+-----------+------------+----------+----+--------+
|senderUsername|senderName|persianYear|persianMonth|persianDay|type|Hashtags|
+--------------+----------+-----------+------------+----------+----+--------+
|mohammad_17799|محمد باجو |1400       |07          |16        |twit|[کنور]  |
+--------------+----------+-----------+------------+----------+----+--------+



...

-------------------------------------------
Batch: 71
-------------------------------------------
+---------------+---------------+-----------+------------+----------+------+--------+
|senderUsername |senderName     |persianYear|persianMonth|persianDay|type  |Hashtags|
+---------------+---------------+-----------+------------+----------+------+--------+
|tavakolizade   |مهدی توکلی زاده|1400       |07          |16        |twit  |[زفکا]  |
|hm8120775447890|گمنام          |1400       |07          |15        |retwit|[دی]    |
|amir2489       |امیر           |1400       |07          |16        |twit  |[غبهار] |
|measam313      |MA313          |1400       |07          |16        |twit  |[برکت]  |
|alis3394       |محمدعلی صادقی  |1400       |07          |16        |quote |[]      |
|matin2930      |اقتصاد فردا    |1400       |07          |16        |twit  |[بترانس]|
|abo09906       |عباس           |1400       |07          |16        |quote |[]      |
+---------------+---------------+---------

-------------------------------------------
Batch: 72
-------------------------------------------
+--------------+----------+-----------+------------+----------+-----+--------+
|senderUsername|senderName|persianYear|persianMonth|persianDay|type |Hashtags|
+--------------+----------+-----------+------------+----------+-----+--------+
|mohammad_17799|محمد باجو |1400       |07          |16        |twit |[کنور]  |
|agooly        |agooly    |1400       |07          |16        |quote|[شلرد]  |
+--------------+----------+-----------+------------+----------+-----+--------+



-------------------------------------------
Batch: 73
-------------------------------------------
+---------------+---------------+-----------+------------+----------+------+--------+
|senderUsername |senderName     |persianYear|persianMonth|persianDay|type  |Hashtags|
+---------------+---------------+-----------+------------+----------+------+--------+
|abo09906       |عباس           |1400       |07          |16        |quote |[]      |
|tavakolizade   |مهدی توکلی زاده|1400       |07          |16        |twit  |[زفکا]  |
|hm8120775447890|گمنام          |1400       |07          |15        |retwit|[دی]    |
|amir2489       |امیر           |1400       |07          |16        |twit  |[غبهار] |
|measam313      |MA313          |1400       |07          |16        |twit  |[برکت]  |
|alis3394       |محمدعلی صادقی  |1400       |07          |16        |quote |[]      |
|matin2930      |اقتصاد فردا    |1400       |07          |16        |twit  |[بترانس]|
|abo09906       |عباس           |1400     

-------------------------------------------
Batch: 74
-------------------------------------------
+--------------+----------+-----------+------------+----------+-----+--------+
|senderUsername|senderName|persianYear|persianMonth|persianDay|type |Hashtags|
+--------------+----------+-----------+------------+----------+-----+--------+
|mohammad_17799|محمد باجو |1400       |07          |16        |twit |[کنور]  |
|agooly        |agooly    |1400       |07          |16        |quote|[شلرد]  |
+--------------+----------+-----------+------------+----------+-----+--------+



-------------------------------------------
Batch: 75
-------------------------------------------
+---------------+---------------+-----------+------------+----------+------+--------+
|senderUsername |senderName     |persianYear|persianMonth|persianDay|type  |Hashtags|
+---------------+---------------+-----------+------------+----------+------+--------+
|abo09906       |عباس           |1400       |07          |16        |quote |[]      |
|tavakolizade   |مهدی توکلی زاده|1400       |07          |16        |twit  |[زفکا]  |
|hm8120775447890|گمنام          |1400       |07          |15        |retwit|[دی]    |
|amir2489       |امیر           |1400       |07          |16        |twit  |[غبهار] |
|measam313      |MA313          |1400       |07          |16        |twit  |[برکت]  |
|alis3394       |محمدعلی صادقی  |1400       |07          |16        |quote |[]      |
|matin2930      |اقتصاد فردا    |1400       |07          |16        |twit  |[بترانس]|
|abo09906       |عباس           |1400     

In [ ]:
# tweetsDF.createOrReplaceTempView("Tweets")

New_DF = spark.sql("SELECT persianYear,persianMonth,persianDay, senderUsername,count(*) as UserActivityCount \
                                    FROM Tweets \
                                    WHERE persianYear = '1400'\
                                    GROUP BY persianYear,persianMonth,persianDay, senderUsername\
                                    ORDER BY persianYear desc,persianMonth desc,persianDay desc,UserActivityCount desc\
                   ")


# Write out our dataframe to the console
query = New_DF.writeStream\
                      .outputMode("complete")\
                      .format("console")\
                      .option("truncate", "false")\
                      .option("numRows", 30)\
                      .start()\
                      .awaitTermination()

### Submit Sample Spark App in Pyspark Container Bash 

- Go to Pyspark Shell :
```bash
docker exec -it pyspark bash
```
- cd /opt/spark-app/
- run this command :
```bash
unset PYSPARK_DRIVER_PYTHON
spark-submit --master  spark-master:7077  --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 sql_queries.py
export PYSPARK_DRIVER_PYTHON=python
```